### Install and import libraries

In [ ]:
!pip install -q s3fs
!pip install -q pyarrow
!pip install -q transformers
!pip install -q torch

In [1]:
import s3fs
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer

### Download the checkpoint model from S3

In [16]:
s3_fs = s3fs.S3FileSystem(
    key = os.environ['AWS_ACCESS_KEY_ID'],
    secret = os.environ['AWS_SECRET_ACCESS_KEY'],
    endpoint_url = os.environ['AWS_S3_ENDPOINT'],
)

In [30]:
files = s3_fs.ls("s3://<Your checkpoint path>")

In [ ]:
os.makedirs('temp', exist_ok=True)

In [31]:
for file in files:
    s3_fs.download(file,'temp')

### Load the model

In [2]:
model_checkpoint = './temp'
model = GPT2LMHeadModel.from_pretrained(model_checkpoint)
tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint, use_fast=True)

### Test the model using a prompt

In [ ]:
model.eval()
max_length = 30
prompt = "There are two pencils in the box. Q: How many pencils? A:"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

outputs = model.generate(input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)